<a href="https://colab.research.google.com/github/Bryan-Az/RL-SARSA-Gym/blob/main/reinforcement_sarsa_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement Learning using SARSA in OpenAI Gym / Gymnasium

## Imports and Install

In [ ]:
import gym

## Step 1: Creating the Environment

In [ ]:
env_name = "Qbert-v0"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def run_gym_env(name,n):
    env = gym.make(name) # Name of the environment supplied as 1st argument
    env.reset()
    for _ in range(int(n)):
        env.render()
        env.step(env.action_space.sample())
    env.close()

In [ ]:
# running the gym env
run_gym_env(env_name, 10)

NameNotFound: Environment Qbert doesn't exist. 

## Step 2: Creating the Agent

In [ ]:
env = gym.make("Qbert-v0")
MAX_NUM_EPISODES = 10
MAX_STEPS_PER_EPISODE = 500
for episode in range(MAX_NUM_EPISODES):
    obs = env.reset()
    for step in range(MAX_STEPS_PER_EPISODE):
        env.render()
        action = env.action_space.sample()# Sample random action. This will be replaced by our agent's action when we start developing the agent algorithms
        next_state, reward, done, info = env.step(action) # Send the action to the environment and receive the next_state, reward and whether done or not
        obs = next_state

        if done is True:
            print("\n Episode #{} ended in {} steps.".format(episode, step+1))
            break

NameNotFound: Environment Qbert doesn't exist. 